In [ ]:
import numpy as np
import scipy as sc

from src.CPQAOA import CP_QAOA
from src.QAOA import QAOA
from src.Pennylane_QAOA import Pennylane_QAOA
from src.Bluequbit_QAOA import Bluequbit_QAOA
from src.Chain import Chain
from src.Grid import Grid
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       normalized_cost, 
                       qubo_limits, 
                       check_qubo, 
                       get_ising)
from src.Tools import get_qiskit_H

In [ ]:
n_qubits = 11
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(n_qubits)])
k = n_qubits // 2
my_indices = [(i, i+1) for i in range(n_qubits-1)]
n_layers = 2
alpha = 0.5
seed = 0

max_iter = 1500

In [ ]:
 # Defining topology
my_chain = Chain(N_qubits=n_qubits)
my_chain.set_initialization_strategy(strategy=init_strat)

# Deciding between grid and 1d chain topology
my_topology = my_chain

# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=n_qubits, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=n_qubits,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
#full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])


if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')

In [ ]:
qiskit_ansatz = QAOA(N_qubits=n_qubits,
                     layers=n_layers,
                     QUBO_matrix=Q,
                     QUBO_offset=offset,
                     constraining_mixer=False,
                     Topology=my_topology)

pennylane_ansatz = Pennylane_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

bluequbit_ansatz = Bluequbit_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

In [ ]:
# Choosing optimizer for scipy
available_methods = ['COBYLA', 'Nelder-Mead', 'BFGS']
optimizer_method = available_methods[0]

# Generating callback function for plotting
qiskit_costs = [] # Normalized costs
qiskit_probs = [] # probability of optimal state
def qiskit_callback(x):
    _dict_ = qiskit_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        qiskit_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        qiskit_probs.append(0)
    qiskit_costs.append(_cost_)
    
# Generating callback function for plotting
pennylane_costs = [] # Normalized costs
pennylane_probs = [] # probability of optimal state
def pennylane_callback(x):
    _dict_ = pennylane_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        pennylane_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        pennylane_probs.append(0)
    pennylane_costs.append(_cost_)
    

# Generating callback function for plotting
bluequbit_costs = [] # Normalized costs
bluequbit_probs = [] # probability of optimal state
def bluequbit_callback(x):
    _dict_ = bluequbit_ansatz.get_state_probabilities(flip_states=False)
    # N.B. Normalizing w. respect to full space max cost
    _cost_ = normalized_cost(result=_dict_,
                             QUBO_matrix=Q,
                             QUBO_offset=offset,
                             max_cost=portfolio_subspace_max_cost, 
                             min_cost=qubo_min_cost)
    if portfolio_subspace_min_state_str in list(_dict_.keys()):
        bluequbit_probs.append(_dict_[portfolio_subspace_min_state_str])
    else:
        bluequbit_probs.append(0)
    bluequbit_costs.append(_cost_)

# Generating initial guess for rotation angles
np.random.seed(seed)
theta_min, theta_max = -2*np.pi, 2*np.pi
N_angles = 2 * n_layers
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_angles)

In [ ]:
qiskit_res = sc.optimize.minimize(fun=qiskit_ansatz.get_cost, 
                                  x0=theta_i,
                                  method=optimizer_method,
                                  options={'disp': False, 
                                           'maxiter': max_iter},
                                  callback=qiskit_callback)
qiskit_res


In [ ]:
pennylane_res = sc.optimize.minimize(fun=pennylane_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter},
                                     callback=pennylane_callback)
pennylane_res

In [ ]:
bluequbit_res = sc.optimize.minimize(fun=bluequbit_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter},
                                     callback=bluequbit_callback)
bluequbit_res

In [ ]:
qiskit_ansatz.circuit_time, qiskit_ansatz.cost_time

In [ ]:
pennylane_ansatz.circuit_time, pennylane_ansatz.cost_time

In [ ]:
bluequbit_ansatz.circuit_time, bluequbit_ansatz.cost_time

In [32]:
import cirq
from typing import Sequence, Tuple, List
import numpy as np
import sympy
import matplotlib.pyplot as plt

from numpy.random import uniform as u

def single_rotation_gamma_layer(gamma_value: float, 
                                h_list: List[Tuple[int, float]]) -> Sequence[cirq.Operation]:
    """ Generator for R_z in U(gamma, C) layer of QAOA """
    for qubit_i, h_i in h_list:
        yield cirq.Z(qubit_i) ** (gamma_value * h_i)
        
def multi_rotation_gamma_layer(gamma_value: float, 
                               J_list: List[Tuple[int, float]]) -> Sequence[cirq.Operation]:
    """ Generator for R_zz in U(gamma, C) layer of QAOA """
    for qubit_i, qubit_j, j_i in J_list:
        yield cirq.ZZ(qubit_i, qubit_j) ** (gamma_value * j_i)

def beta_layer(beta_value: float, indices: list[int]) -> Sequence[cirq.Operation]:
    """Generator for U(beta, B) layer (mixing layer) of QAOA"""
    for qubit_i in indices:
        yield cirq.X(qubit_i) ** beta_value
        
layers = 2
N_qubits = 15
qubits = [cirq.NamedQubit(f'q_{i}') for i in range(N_qubits)]

h_list = [(qubits[i],u(-2,2)) for i in range(N_qubits)]
J_list = [(qubits[i],qubits[i+1],u(-2,2)) for i in range(N_qubits-1)]

# Use sympy.Symbols for the 𝛾 and β parameters.
gammas = [sympy.Symbol(f"𝛄_{layer}") for layer in range(layers)]
betas = [sympy.Symbol(f"β_{layer}") for layer in range(layers)]

# Start in the H|0> state.
qaoa = cirq.Circuit(cirq.H.on_each(qubits))
for layer in range(layers):
    
    # Implement the U(gamma, C) operator.
    qaoa.append(single_rotation_gamma_layer(gamma_value=gammas[layer],h_list=h_list))
    qaoa.append(multi_rotation_gamma_layer(gamma_value=gammas[layer], J_list=J_list))
    
    # Implement the U(beta, B) operator.
    qaoa.append(beta_layer(betas[layer], indices=qubits), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

gamma_values, beta_values = [u(-4,4) for gamma in gammas], [u(-4,4) for beta in betas]
sim = cirq.Simulator()
gamma_dict, beta_dict = {f"𝛄_{i}": gamma_values[i] for i in range(len(gammas))}, {f"β_{i}": beta_values[i] for i in range(len(gammas))}
total_dict = {**gamma_dict, **beta_dict}
params = cirq.ParamResolver(total_dict)
wf = sim.simulate(qaoa, param_resolver=params).final_state_vector